In [1]:
# kd crossover，close < ema 8, ema 8 < ema 18, ema 18 < 38, take profit 4 atr, stop loss 3 atr
# ! conda install -c conda-forge ta --yes

In [2]:
import requests
import pandas as pd
import ta
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import time

In [3]:
timezone = 8
endpoint = 'wss://stream.binance.com:9443/ws'
symbol = 'ethusdt'
symbol_C = symbol.upper()
interval = '15m'

# start epoch till now
start_time = 1672564611000
end_time = round(time.time() * 1000)
# end_time = 1672556207000

# step between timestamps in milliseconds
step = 60000 * 3600

In [4]:

sl_atr = 3
tp_atr = 6
kd_dir = 'Up'

In [5]:
def get_historical(symbol, interval, start_time, end_time, step):
    
    raw_df = pd.DataFrame()
    
    url = "https://api.binance.com/api/v3/klines"
    
    for timestamp in range(start_time, end_time, step):
        params = {"symbol": symbol_C,
                  "interval": interval,
                  "startTime": timestamp,
                  "endTime": timestamp + step}
        response = requests.get(url, params=params).json()
        out = pd.DataFrame(response, columns = ["Open time", "Open", "High", "Low", "Close",
                                               "Volume", "Close_Time", "Quote asset volume",
                                               "Number of trades", "Taker buy base asset volume",
                                               "Taker buy quote asset volume", "Ignore"])
        raw_df = pd.concat([raw_df, out], axis = 0)
    
    raw_df = raw_df[['Close_Time', 'Open', 'Close', "High", "Low", 'Volume']]
    convert_dict = {'Close_Time': float, 'Open': float, 'Close': float, "High": float, "Low": float, 'Volume': float}
    raw_df = raw_df.astype(convert_dict)
    
    raw_df['Close_Time'] = pd.to_datetime(raw_df['Close_Time'], unit = 'ms')
    raw_df['Close_Time'] = raw_df['Close_Time'] + pd.Timedelta(hours=timezone)
    raw_df['Close_Time'] = raw_df['Close_Time'].dt.strftime('%Y-%m-%d %H:%M:%S')
    
    raw_df = raw_df.reset_index(drop=True)
    
    return raw_df

In [6]:
raw_df = get_historical(symbol, interval, start_time, end_time, step)
print(raw_df)

               Close_Time     Open    Close     High      Low     Volume
0     2023-01-01 17:44:59  1194.76  1195.98  1195.99  1194.76   450.9908
1     2023-01-01 17:59:59  1195.98  1196.48  1196.55  1195.60   738.0895
2     2023-01-01 18:14:59  1196.47  1196.29  1196.48  1195.70   489.0501
3     2023-01-01 18:29:59  1196.29  1196.21  1196.34  1195.98   380.1043
4     2023-01-01 18:44:59  1196.22  1195.66  1196.22  1195.59   556.5141
...                   ...      ...      ...      ...      ...        ...
8756  2023-04-02 23:59:59  1795.32  1797.54  1798.70  1793.69  5114.2826
8757  2023-04-03 00:14:59  1797.54  1796.16  1798.69  1795.26  3488.1824
8758  2023-04-03 00:29:59  1796.16  1792.99  1796.56  1792.31  4303.7175
8759  2023-04-03 00:44:59  1793.00  1793.68  1795.82  1785.12  8447.5004
8760  2023-04-03 00:59:59  1793.68  1795.28  1795.90  1792.20  5029.0726

[8761 rows x 6 columns]


In [7]:
df = raw_df.copy()

In [8]:
def check_cross(df, kd_dir):
    up = df['slow_k'] > df['slow_d']
    down = df['slow_k'] < df['slow_d']
    if kd_dir == 'Up':
        return up.diff() & up
    if kd_dir == 'Any':
        return up.diff()
    if kd_dir == 'Down':
        return down.diff() & down


def indicators(df, kd_dir):

# ema
    for i in (8, 18, 38):
        df['ema_'+str(i)] = ta.trend.ema_indicator(df.Close, window=i)

# atr
    df['atr'] = ta.volatility.average_true_range(df.High, df.Low, df.Close)
    
# rsi
    rsi_int = 14
    df['rsi'] = ta.momentum.RSIIndicator(df['Close'], window = rsi_int).rsi()

# kd
    kd_int = 14
    d_int = 3
   
    kd_df = pd.DataFrame()
    kd_df[str(kd_int) + '-Low'] = df['Low'].rolling(kd_int).min()
    kd_df[str(kd_int) + '-High'] = df['High'].rolling(kd_int).max()
    df['slow_k'] = (df['Close'] - kd_df[str(kd_int) + '-Low'])*100/(kd_df[str(kd_int) + '-High'] - kd_df[str(kd_int) + '-Low'])
    df['slow_d'] = df['slow_k'].rolling(d_int).mean()
    
# kd cross
    df['kd_cross'] = check_cross(df, kd_dir)

In [9]:
indicators(df, kd_dir)
print(df)

               Close_Time     Open    Close     High      Low     Volume  \
0     2023-01-01 17:44:59  1194.76  1195.98  1195.99  1194.76   450.9908   
1     2023-01-01 17:59:59  1195.98  1196.48  1196.55  1195.60   738.0895   
2     2023-01-01 18:14:59  1196.47  1196.29  1196.48  1195.70   489.0501   
3     2023-01-01 18:29:59  1196.29  1196.21  1196.34  1195.98   380.1043   
4     2023-01-01 18:44:59  1196.22  1195.66  1196.22  1195.59   556.5141   
...                   ...      ...      ...      ...      ...        ...   
8756  2023-04-02 23:59:59  1795.32  1797.54  1798.70  1793.69  5114.2826   
8757  2023-04-03 00:14:59  1797.54  1796.16  1798.69  1795.26  3488.1824   
8758  2023-04-03 00:29:59  1796.16  1792.99  1796.56  1792.31  4303.7175   
8759  2023-04-03 00:44:59  1793.00  1793.68  1795.82  1785.12  8447.5004   
8760  2023-04-03 00:59:59  1793.68  1795.28  1795.90  1792.20  5029.0726   

            ema_8       ema_18       ema_38       atr        rsi     slow_k  \
0       

In [10]:
def conditions(df):

    for index, row in df.iterrows():
        # c1
        df['c1'] = df['kd_cross']
        # c2
        df['c2'] = df['Close'] <= df['ema_8']
        # c3
        df['c3'] = df['ema_8'] <= df['ema_18']
        # c4
        df['c4'] = df['ema_18'] <= df['ema_38']
        
        df['c5'] = df['rsi'] >= 30


    # 條件達成
    df['signal'] = False
    df.loc[df.c1 & df.c2 & df.c3 & df.c4 & df.c5, 'signal'] = True


    # 下一根進場
    df['open_entry'] = False
    for i in range(len(df) - 1):
        if df.loc[i, 'signal'] == True:
            df.loc[i + 1, 'open_entry'] = True
#     print(df[df['Close_Time'].str.contains('2023-03-11 16')])
    
    return df

In [11]:
conditions(df)

,Close_Time,Open,Close,High,Low,Volume,ema_8,ema_18,ema_38,atr,...,slow_k,slow_d,kd_cross,c1,c2,c3,c4,c5,signal,open_entry
0,2023-01-01 17:44:59,1194.76,1195.98,1195.99,1194.76,450.9908,NaN,NaN,NaN,0.000000,...,NaN,NaN,False,False,False,False,False,False,False,False
1,2023-01-01 17:59:59,1195.98,1196.48,1196.55,1195.60,738.0895,NaN,NaN,NaN,0.000000,...,NaN,NaN,False,False,False,False,False,False,False,False
2,2023-01-01 18:14:59,1196.47,1196.29,1196.48,1195.70,489.0501,NaN,NaN,NaN,0.000000,...,NaN,NaN,False,False,False,False,False,False,False,False
3,2023-01-01 18:29:59,1196.29,1196.21,1196.34,1195.98,380.1043,NaN,NaN,NaN,0.000000,...,NaN,NaN,False,False,False,False,False,False,False,False
4,2023-01-01 18:44:59,1196.22,1195.66,1196.22,1195.59,556.5141,NaN,NaN,NaN,0.000000,...,NaN,NaN,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8756,2023-04-02 23:59:59,1795.32,1797.54,1798.70,1793.69,5114.2826,1802.794116,1808.387688,1812.711888,5.309589,...,27.393703,22.265498,False,False,True,True,True,True,False,False
8757,2023-04-03 00:14:59,1797.54,1796.16,1798.69,1795.26,3488.1824,1801.319868,1807.100563,1811.863073,5.175332,...,22.914638,23.509683,False,False,True,True,True,False,False,False
8758,2023-04-03 00:29:59,1796.16,1792.99,1796.56,1792.31,4303.7175,1799.468786,1805.615241,1810.895223,5.109237,...,12.625771,20.978037,False,False,True,True,True,False,False,False
8759,2023-04-03 00:44:59,1793.00,1793.68,1795.82,1785.12,8447.5004,1798.182389,1804.358900,1810.012392,5.508577,...,24.604771,20.048393,True,True,True,True,True,False,False,False


In [12]:
entry_df = conditions(df)

# 進出場

in_position = False
stop_loss = np.nan
take_profit = np.nan
close_val = entry_df['Close']
atr_val = entry_df['atr']
open_val = entry_df['Open']


for index, row in entry_df.iterrows():

    if index == 0:
        continue
    
    elif entry_df.at[index, 'open_entry'] == True:
   
        entry_df.at[index, 'entry_p'] = close_val.shift(1).at[index]
        entry_df.at[index, 'stop_loss'] = close_val.shift(1).at[index] + sl_atr * atr_val.shift(1).at[index]
        entry_df.at[index, 'take_profit'] = close_val.shift(1).at[index] - tp_atr * atr_val.shift(1).at[index]
        entry_df.at[index, 'position'] = 'Short'
        in_position = True
        stop_loss = entry_df.at[index, 'stop_loss']
        take_profit = entry_df.at[index, 'take_profit']
    

    # 吃筍
    #-----------------------------重要-----------------------------
    # 若用 if 寫，則有可能入場馬上吃筍，若用 elif 則一個 iteration 只會執行一次
    elif in_position == True and (entry_df.at[index, 'Close'] >= stop_loss):
        entry_df.at[index, 'position'] = 'Stop'
        in_position = False
        stop_loss = np.nan
        take_profit = np.nan

    # set take profit
    elif in_position == True and (entry_df.at[index, 'Low'] <= take_profit):
        entry_df.at[index, 'position'] = 'Buy'
        in_position = False
        stop_loss = np.nan
        take_profit = np.nan


entry_df.to_csv('entry_df.csv')
        
raw_entry_df = entry_df.copy()

# 過濾有訊號或事件發生的Ｋ線
entry_df = entry_df[(entry_df['open_entry'] == True) |
              (entry_df['signal'] == True) | 
              (entry_df['position'] == 'Short') |
              (entry_df['position'] == 'Buy') |
              (entry_df['position'] == 'Stop')]


print(entry_df)


               Close_Time     Open    Close     High      Low      Volume  \
63    2023-01-02 09:29:59  1195.75  1196.09  1196.13  1193.00   3685.1617   
64    2023-01-02 09:44:59  1196.09  1196.03  1196.44  1195.27    953.3041   
66    2023-01-02 10:14:59  1195.15  1196.26  1196.41  1194.46   1355.2923   
67    2023-01-02 10:29:59  1196.27  1196.20  1196.42  1195.55    608.6620   
73    2023-01-02 11:59:59  1197.72  1201.94  1203.53  1197.40   5749.0388   
...                   ...      ...      ...      ...      ...         ...   
8743  2023-04-02 20:44:59  1812.80  1815.42  1815.54  1810.20   3978.8460   
8744  2023-04-02 20:59:59  1815.43  1813.30  1815.43  1811.86   2424.6461   
8746  2023-04-02 21:29:59  1809.20  1812.35  1812.55  1805.01   7101.3629   
8747  2023-04-02 21:44:59  1812.35  1815.68  1819.91  1811.72   8678.3901   
8754  2023-04-02 23:29:59  1796.56  1795.01  1798.29  1789.10  11127.2390   

            ema_8       ema_18       ema_38       atr  ...     c2    c3    

In [13]:
# 部位回測

pos_df = entry_df.copy()
pos_df = pos_df.reset_index(drop = True)
pos_df = pos_df[(pos_df['position'] == 'Short') |
              (pos_df['position'] == 'Buy') |
              (pos_df['position'] == 'Stop')]

# 一次進場多少單位
pos_size = 1

col = ['Close_Time', 'Open', 'Close', 'High', 'Low', 'ema_8', 'ema_18', 'ema_38', 'atr', 'kd_cross', 'position','entry_p', 'stop_loss', 'take_profit']
pos = pos_df[col]
pos = pos.reset_index(drop = True)


for index, row in pos.iterrows():
    
    current_pos = 0
    
    # 進場
    if pos.at[index, 'position'] == 'Short':
        pos.at[index, 'size'] = pos_size
        pos.exit_p = np.nan
    
    # 出場
    if pos.at[index, 'position'] == 'Buy' or pos.at[index, 'position'] == 'Stop':
        
        #-----------------------------重要-----------------------------
        # 實戰需即刻出場
        
        # 停利：達成條件時收盤價
        if pos.at[index, 'position'] == 'Buy':
            for i in range(index -1, -1, -1):
                if pos.at[i, 'position'] == 'Short':
                    pos.at[index, 'exit_p'] = pos.at[i, 'take_profit']
                break

        # 停損：打到進場停損點（往回跌代，直到最近的'Buy'及其'stop_loss'）
        if pos.at[index, 'position'] == 'Stop':
            for i in range(index -1, -1, -1):
                if pos.at[i, 'position'] == 'Short':
                    pos.at[index, 'exit_p'] = pos.at[i, 'stop_loss']
                break

        # 計算每次出場部位大小（每次出場皆清倉）
        for i in range(index -1, -1, -1):
            if pos.at[i, 'position'] == 'Short':
                current_pos += pos.at[i, 'size']
                if i == 0:
                    pos.at[index, 'size'] = -current_pos
                else:
                    continue
            else:
                pos.at[index, 'size'] = -current_pos
                current_pos = 0
                break


# 計算部位價值
for index, row in pos.iterrows():
    if pos.at[index, 'position'] == 'Short':
        pos.at[index, 'amt'] = round(pos.at[index, 'size'] * pos.at[index, 'entry_p'], 4)
    elif pos.at[index, 'position'] == 'Buy' or pos.at[index, 'position'] == 'Stop':
        pos.at[index, 'amt'] = round(pos.at[index, 'size'] * pos.at[index, 'exit_p'], 4)


# 若最後一筆為 Buy，移除該單，迭代驗證
for index, row in pos.iloc[::-1].iterrows():
    if row['position'] == 'Short':
        pos = pos.drop(index)
    else:
        break


# 手續費、滑點、價差
fee = 0.05 / 100
amt_abs_sum = pos.amt.abs().sum()
ttl_fee = amt_abs_sum * fee

# 損益
leverage = 10
ttl_profit = pos.amt.sum() - ttl_fee


# 計算進場最大部位，最大損益
consec_entry = 0
position_amt_sum = 0
max_consec_entry = 0
max_position = 0
max_profit = 0
max_loss = 0

for index, row in pos.iterrows():
    
    if row['position'] == 'Short':
        
        consec_entry += 1
        position_amt_sum += row['amt']
        
    elif row['position'] in ['Buy', 'Stop']:
        
        if consec_entry > max_consec_entry:
            max_consec_entry = consec_entry
            max_position = position_amt_sum
            
        position_amt_sum += row['amt']
        
        if position_amt_sum > max_profit:
            max_profit = position_amt_sum
            
        if position_amt_sum < max_loss:
            max_loss = position_amt_sum
            
        consec_entry = 0
        position_amt_sum = 0
    
    else:
        pass


profit_per = "{:.2f}%".format(ttl_profit / (max_position/leverage) * 100)

wins = pos['position'].str.count('Buy').sum()
loses = pos['position'].str.count('Stop').sum()

win_rate = "{:.2f}%".format(wins / (wins + loses) * 100)

result = {'Profit': [round(ttl_profit, 2)],
          'Fee': [round(ttl_fee, 2)],
          'Max_Profit': [round(max_profit, 2)],
          'Max_Loss': [round(max_loss, 2)],
          'Max_Entry': [max_consec_entry],
          'Max_Position': [round(max_position, 2)],
          'Profit_%': [profit_per],
          'Win_Rate': [win_rate]}


result_df = pd.DataFrame(result)

print(result_df)
print()
print(pos)

pos.to_csv('pos.csv')



    Profit    Fee  Max_Profit  Max_Loss  Max_Entry  Max_Position Profit_%  \
0  1084.25  581.8      843.97   -150.78         14       21581.6   50.24%   

  Win_Rate  
0   21.18%  

              Close_Time     Open    Close     High      Low        ema_8  \
0    2023-01-02 09:44:59  1196.09  1196.03  1196.44  1195.27  1197.635326   
1    2023-01-02 10:29:59  1196.27  1196.20  1196.42  1195.55  1196.745894   
2    2023-01-02 11:59:59  1197.72  1201.94  1203.53  1197.40  1198.072030   
3    2023-01-03 09:59:59  1213.56  1214.14  1214.27  1212.82  1214.332853   
4    2023-01-03 20:29:59  1215.34  1215.92  1216.39  1215.01  1215.666673   
..                   ...      ...      ...      ...      ...          ...   
447  2023-04-02 14:59:59  1822.05  1823.09  1824.81  1822.05  1820.296346   
448  2023-04-02 20:29:59  1817.05  1812.80  1817.05  1811.20  1817.101227   
449  2023-04-02 20:59:59  1815.43  1813.30  1815.43  1811.86  1815.965928   
450  2023-04-02 21:44:59  1812.35  1815.68  1819

PermissionError: [Errno 13] Permission denied: 'pos.csv'

In [ ]:
plt_df = raw_entry_df.copy()

plt.figure(figsize=(80, 36))
plt.ylabel('Close')
plt.plot(plt_df.Close, label = 'Close', c ='black')
for i in range(len(plt_df)):
    if plt_df.position[i] == 'Buy':
        plt.plot(i, plt_df.entry_p[i], 'go', label='_', markersize=20)
    elif plt_df.position[i] == 'Stop':
        plt.plot(i, plt_df.entry_p[i], 'ro', label='_', markersize=20)
# plt.plot(plt_df.ema_8, label='ema_8', c='g', lw=0.5)
# plt.plot(plt_df.ema_14, label='ema_14', c='r', lw=0.5)
# plt.plot(plt_df.ema_50, label='ema_50', c='b', lw=0.5)
# plt.plot(plt_df.ema_200, label='ema_200', c='y', lw=4)
plt.legend()
plt.show()